https://teddylee777.github.io/huggingface/bert-kor-text-classification/

### Module

In [1]:
# pip install Korpora

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 1.1 MB/s eta 0:00:00


In [21]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# 한글 자연어 처리 데이터셋
from Korpora import Korpora

# 토크나이저 관련 경고 무시하기 위하여 설정
os.environ["TOKENIZERS_PARALLELISM"] = 'true'

# device 지정
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'사용 디바이스: {device}')

corpus = Korpora.load("nsmc")

사용 디바이스: cuda:0

    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at /root/Korpora/nsmc/ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at /root

In [17]:
# torch.cuda.is_available()
CUDA_VISIBLE_DEVICES

NameError: ignored

### Data

In [ ]:
# data = pd.read_csv('data/data_5474.csv')
# data
# data['3'][5]

,0,1,2,3,4,5
0,전문가가 알려주는 창의 영재교육의 비법,알차네요,star star-rate5,무엇보다도 빨리들을 수 있어 좋습니다,학습지도,2023.07.28
1,연수로 완성하는 아이스크림 수학 교과서(4학년),명불허전,star star-rate4,명강사라 그런지 말씀을 참 잘하시네요~,교과지도,2023.07.28
2,본격! 청개구리 학생심리 사로잡기,강추,star star-rate5,30년 가까이 일하면서도 놓치고 있었던 것들을 알려주었습니다. 그리고 무조건 교사 ...,학급경영,2023.07.28
3,"가장 쉬운 수업도구, 패들렛과 띵커벨",패들렛과 띵커벨 활용 잘 할 수 있어요,star star-rate5,패들렛과 띵커벨 활용하고 있었으나 다양한 교과에 특히 독서 지도에 많이 활용 할 수...,ICT,2023.07.28
4,신영일과 함께하는 안전교육 365,필수 연수 부담 없이 편안하게,star star-rate4,"안전 연수 올 해 꼭 받아야 하는데, 부담 없이 편안한 마음으로 마쳤어요.~",생활지도,2023.07.27
...,...,...,...,...,...,...
5352,알베르토가 함께 하는 핵심역량 연계 다문화 교육,알베르토가 함께하는 핵심역량 연계 다문화 교육,star star-rate5,여타 연수와 다르게 새로 알게된 내용이 많이 있습니다.\n아이들 가르치는데 많은 도...,생활지도,2022.03.30
5353,전문가가 알려주는 창의 영재교육의 비법,전문가가 알려주는 창의 영재교육의 비법 연수 후기,star star-rate5,영재 교육에 대한 틀을 잡아주시는 넘 좋은 연수였습니다!\n많은 도움이 되어 강력 ...,학습지도,2022.03.30
5354,도도한 교사생활,도도한 교사생활,star star-rate5,연수를 통해 자신감을 더 갖게 되었다.,생활지도,2022.03.30
5355,생각을 코딩하다! 소프트웨어교실,교실에서 활용할 수 있는 앱들 중심으로 재미있게 들었어요.,star star-rate5,교실에서 활용할 수 있는 앱들 중심으로 재미있게 들었어요.\n코로나19로 집콕하며 ...,ICT,2022.03.30


In [3]:
train = pd.read_csv('~/Korpora/nsmc/ratings_train.txt', sep='\t')
test = pd.read_csv('~/Korpora/nsmc/ratings_test.txt', sep='\t')

In [4]:
# 문장 길이 계산
train['length'] = train['document'].apply(lambda x: len(str(x)))
test['length'] = test['document'].apply(lambda x: len(str(x)))

너무 짧은 문장은 식별하기 어려워 제외

In [5]:
train = train.loc[train['length'] > 5]

# 전체 데이터셋 크기가 커서 1000개의 문장을 샘플링 합니다.
train = train.sample(1000)
train

,id,document,label,length
96910,1540092,최고 첨으로 시디에 구운 애니메이션 정말 최고~,1,26
19663,6594701,꼬마 참 야무지네...,0,12
56867,6176089,방법과 컨텐츠 모두 와 닿지가 않네요. 연출과 각본 결국 감독의 문제겠죠,0,40
61769,10221269,예쁜 레아 세이두와 다양한 의상을 구경하는 재미 빼면 남는게 하나도 없는 영화.떡밥...,0,101
23341,9773961,"모임이 있어서 8, 9살 딸과 아들에게 영화보고 있으라고 오세암을 보여주고 외출했다...",1,133
...,...,...,...,...
52515,2956560,시사회 보고 왔습니다... 영화를 만들다 만거니??? 너무 기대를 하고 봤나보다.. ㅠㅠ,0,49
116285,9135730,"새벽감성이 달콤하고 짙게 베인 영화. 열심히 살다보면, 잊혀지고 또 얻게 되겠지.",1,45
90924,7346936,너무너무 따듯한 영화^3^마음이 녹아용,1,21
114767,8776760,너무 답답하고 짜즈히나요 계속 빙빙 돌리기만 하고 상식적으로 이 유라는 너무 몰상식...,0,53


### 토큰화 적용된 데이터셋

한글 데이터 셋으로 사전학습된 모델 지정
<br>'kykil/bert-kor-base' 사용
<br>https://huggingface.co/kykim/bert-kor-base

In [6]:
CHECKPOINT_NAME = 'kykim/bert-kor-base'

In [8]:
# pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.5 MB/s eta 0:00:00


In [9]:
import torch
from transformers import BertTokenizerFast
from torch.utils.data import Dataset, DataLoader


class TokenDataset(Dataset):

    def __init__(self, dataframe, tokenizer_pretrained):
        # sentence, label 컬럼으로 구성된 데이터프레임 전달
        self.data = dataframe
        # Huggingface 토크나이저 생성
        self.tokenizer = BertTokenizerFast.from_pretrained(tokenizer_pretrained)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence = self.data.iloc[idx]['document']
        label = self.data.iloc[idx]['label']

        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )

        input_ids = tokens['input_ids'].squeeze(0)           # 2D -> 1D
        attention_mask = tokens['attention_mask'].squeeze(0) # 2D -> 1D
        token_type_ids = torch.zeros_like(attention_mask)

        # input_ids, attention_mask, token_type_ids 이렇게 3가지 요소를 반환하도록 합니다.
        # input_ids: 토큰
        # attention_mask: 실제 단어가 존재하면 1, 패딩이면 0 (패딩은 0이 아닐 수 있습니다)
        # token_type_ids: 문장을 구분하는 id. 단일 문장인 경우에는 전부 0
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids,
        }, torch.tensor(label)

nn.Dataset을 확장한 클래스인 TokenDataset 클래스 인스턴스 생성

In [10]:
# 토크나이저 지정
tokenizer_pretrained = CHECKPOINT_NAME

# train, test 데이터셋 생성
train_data = TokenDataset(train, tokenizer_pretrained)
test_data = TokenDataset(test, tokenizer_pretrained)

# DataLoader로 이전에 생성한 Dataset을 지정하여, batch, shuffle, num_workers 등 설정
train_loader = DataLoader(train_data, batch_size=8, shuffle=True, num_workers=8)
test_loader = DataLoader(test_data, batch_size=8, shuffle=True, num_workers=8)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


생성된 train_loader로부터 1개의 batch만 꺼내보기

In [22]:
# 1개의 batch 꺼내기
inputs, labels = next(iter(train_loader))

# 데이터셋 device 설정
inputs = {k: v.to(device) for k, v in inputs.items()}
labels.to(device)

tensor([1, 0, 0, 0, 1, 1, 0, 1], device='cuda:0')

In [23]:
# 생성된 inputs의 key 값 출력
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'token_type_ids'])

input_ids: 토큰

attention_mask: 실제 단어가 존재하면 1, 패딩이면 0 (패딩은 0이 아닐 수 있습니다)

token_type_ids: 문장을 구분하는 id. 단일 문장인 경우에는 전부 0

In [24]:
# key 별 shape 확인
inputs['input_ids'].shape, inputs['attention_mask'].shape, inputs['token_type_ids'].shape

(torch.Size([8, 512]), torch.Size([8, 512]), torch.Size([8, 512]))

batch_size = 8 (학습 사이즈)

1개 문장의 최대 길이 : 512

아래 코드를 통해 'max_position_embeddings'에 512가 할당되어 있는 것을 알 수 있다

In [26]:
from transformers import BertConfig

config = BertConfig.from_pretrained(CHECKPOINT_NAME)
config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "embedding_size": 768,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 42000
}

labels는 one-hot encoding 되지 않은 상태로 출력

In [27]:
# labels 출력
labels

tensor([1, 0, 0, 0, 1, 1, 0, 1])

이전에 생성한 inputs를 한 번 대입해본 후 결과 확인

In [ ]:
output = model_bert(**inputs)
output.keys()

### Pre-Trained Model 생성

Pre-Trained BERT Model은 BertModel의 from_pretrained 함수로 간단히 가져올 수 있음

In [ ]:
from transformers import BertModel

# 모델 생성
model_bert = BertModel.from_pretrained(CHECKPOINT_NAME).to(device)
model_bert

이전에 생성한 inputs 대입한 후 결과 확인

In [ ]:
output = model_bert(**inputs)
output.keys()

In [ ]:
output['last_hidden_state'].shape, output['pooler_output'].shape

In [ ]:
# last_hidden_state 출력
last_hidden_state = output['last_hidden_state']
print(last_hidden_state.shape)
print(last_hidden_state[:, 0, :])

In [ ]:
# pooler_output 출력
pooler_output = output['pooler_output']
print(pooler_output.shape)
print(pooler_output)

output으로 나온 last_hidden_state[:, 0, :]의 [CLS] 토큰을 가져온 뒤 FC에 입력으로 대입
<br>Binary-Classificition을 위하여 최종 출력 값은 2로 설정

<br>결과 shape은 [8, 2] 입니다. 8개의 batch 에 대해 2개의 출력

In [ ]:
fc = nn.Linear(768, 2)
fc.to(device)
fc_output = fc(last_hidden_state[:, 0, :])
print(fc_output.shape)
print(fc_output.argmax(dim=1))

### 사전학습(pre-trained) BERT 모델을 활용한 텍스트 분류 모델 생성

In [ ]:
class CustomBertModel(nn.Module):
    def __init__(self, bert_pretrained, dropout_rate=0.5):
        # 부모클래스 초기화
        super(CustomBertModel, self).__init__()
        # 사전학습 모델 지정
        self.bert = BertModel.from_pretrained(bert_pretrained)
        # dropout 설정
        self.dr = nn.Dropout(p=dropout_rate)
        # 최종 출력층 정의
        self.fc = nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        # 입력을 pre-trained bert model 로 대입
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # 결과의 last_hidden_state 가져옴
        last_hidden_state = output['last_hidden_state']
        # last_hidden_state[:, 0, :]는 [CLS] 토큰을 가져옴
        x = self.dr(last_hidden_state[:, 0, :])
        # FC 을 거쳐 최종 출력
        x = self.fc(x)
        return x

In [ ]:
# CustomBertModel 생성
bert = CustomBertModel(CHECKPOINT_NAME)
bert.to(device)

In [ ]:
# loss 정의: CrossEntropyLoss
loss_fn = nn.CrossEntropyLoss()

# 옵티마이저 정의: bert.paramters()와 learning_rate 설정
optimizer = optim.Adam(bert.parameters(), lr=1e-5)

In [ ]:
from tqdm import tqdm  # Progress Bar 출력

def model_train(model, data_loader, loss_fn, optimizer, device):
    # 모델을 훈련모드로 설정합니다. training mode 일 때 Gradient 가 업데이트 됩니다. 반드시 train()으로 모드 변경을 해야 합니다.
    model.train()

    # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
    running_loss = 0
    corr = 0
    counts = 0

    # 예쁘게 Progress Bar를 출력하면서 훈련 상태를 모니터링 하기 위하여 tqdm으로 래핑합니다.
    prograss_bar = tqdm(data_loader, unit='batch', total=len(data_loader), mininterval=1)

    # mini-batch 학습을 시작합니다.
    for idx, (inputs, labels) in enumerate(prograss_bar):
        # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
        inputs = {k:v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)

        # 누적 Gradient를 초기화 합니다.
        optimizer.zero_grad()

        # Forward Propagation을 진행하여 결과를 얻습니다.
        output = model(**inputs)

        # 손실함수에 output, label 값을 대입하여 손실을 계산합니다.
        loss = loss_fn(output, labels)

        # 오차역전파(Back Propagation)을 진행하여 미분 값을 계산합니다.
        loss.backward()

        # 계산된 Gradient를 업데이트 합니다.
        optimizer.step()

        # output의 max(dim=1)은 max probability와 max index를 반환합니다.
        # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
        _, pred = output.max(dim=1)

        # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
        # 합계는 corr 변수에 누적합니다.
        corr += pred.eq(labels).sum().item()
        counts += len(labels)

        # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
        # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
        # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
        running_loss += loss.item() * labels.size(0)

        # 프로그레스바에 학습 상황 업데이트
        prograss_bar.set_description(f"training loss: {running_loss/(idx+1):.5f}, training accuracy: {corr / counts:.5f}")

    # 누적된 정답수를 전체 개수로 나누어 주면 정확도가 산출됩니다.
    acc = corr / len(data_loader.dataset)

    # 평균 손실(loss)과 정확도를 반환합니다.
    # train_loss, train_acc
    return running_loss / len(data_loader.dataset), acc

In [ ]:
def model_evaluate(model, data_loader, loss_fn, device):
    # model.eval()은 모델을 평가모드로 설정을 바꾸어 줍니다.
    # dropout과 같은 layer의 역할 변경을 위하여 evaluation 진행시 꼭 필요한 절차 입니다.
    model.eval()

    # Gradient가 업데이트 되는 것을 방지 하기 위하여 반드시 필요합니다.
    with torch.no_grad():
        # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
        corr = 0
        running_loss = 0

        # 배치별 evaluation을 진행합니다.
        for inputs, labels in data_loader:
            # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
            inputs = {k:v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)

            # 모델에 Forward Propagation을 하여 결과를 도출합니다.
            output = model(**inputs)

            # output의 max(dim=1)은 max probability와 max index를 반환합니다.
            # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
            _, pred = output.max(dim=1)

            # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
            # 합계는 corr 변수에 누적합니다.
            corr += torch.sum(pred.eq(labels)).item()

            # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
            # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
            # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
            running_loss += loss_fn(output, labels).item() * labels.size(0)

        # validation 정확도를 계산합니다.
        # 누적한 정답숫자를 전체 데이터셋의 숫자로 나누어 최종 accuracy를 산출합니다.
        acc = corr / len(data_loader.dataset)

        # 결과를 반환합니다.
        # val_loss, val_acc
        return running_loss / len(data_loader.dataset), acc

In [ ]:
# 최대 Epoch을 지정합니다.
num_epochs = 10

# checkpoint로 저장할 모델의 이름을 정의 합니다.
model_name = 'bert-kor-base'

min_loss = np.inf

# Epoch 별 훈련 및 검증을 수행합니다.
for epoch in range(num_epochs):
    # Model Training
    # 훈련 손실과 정확도를 반환 받습니다.
    train_loss, train_acc = model_train(bert, train_loader, loss_fn, optimizer, device)

    # 검증 손실과 검증 정확도를 반환 받습니다.
    val_loss, val_acc = model_evaluate(bert, test_loader, loss_fn, device)

    # val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장합니다.
    if val_loss < min_loss:
        print(f'[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss
        torch.save(bert.state_dict(), f'{model_name}.pth')

    # Epoch 별 결과를 출력합니다.
    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')

### 예시 문장을 입력하여 긍/부정 예측하기

In [ ]:
# 저장한 state_dict를 로드 합니다.
bert.load_state_dict(torch.load(f'{model_name}.pth'))

In [ ]:
class CustomPredictor():
    def __init__(self, model, tokenizer, labels: dict):
        self.model = model
        self.tokenizer = tokenizer
        self.labels = labels

    def predict(self, sentence):
        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )
        tokens.to(device)
        prediction = self.model(**tokens)
        prediction = F.softmax(prediction, dim=1)
        output = prediction.argmax(dim=1).item()
        prob, result = prediction.max(dim=1)[0].item(), self.labels[output]
        print(f'[{result}]\n확률은: {prob*100:.3f}% 입니다.')

In [ ]:
# Huggingface 토크나이저 생성
tokenizer = BertTokenizerFast.from_pretrained(CHECKPOINT_NAME)

labels = {
    0: '부정 리뷰 입니다.',
    1: '긍정 리뷰 입니다.'
}

# CustomPredictor 인스턴스를 생성합니다.
predictor = CustomPredictor(bert, tokenizer, labels)

In [ ]:
# 사용자 입력에 대하여 예측 후 출력을 낼 수 있는 간단한 함수를 생성합니다.
def predict_sentence(predictor):
    input_sentence = input('문장을 입력해 주세요: ')
    predictor.predict(input_sentence)

In [ ]:
# 부정 리뷰 입력 예시
predict_sentence(predictor)